# Load modules

In [ ]:
from pyiron_base import load
import sys
sys.path.insert(0,'/cmmc/u/cfrey/devel/python-test')
import EXTRA_FIM.main as fim
from EXTRA_FIM.sx_nc_waves_reader import sx_nc_waves_reader
from EXTRA_FIM.potential import extend_potential, sx_el_potential3D_cell

# Get simulation parameters

In [ ]:
# Get parameter dict from pyiron job
Simulator = load()

# Extend the potential if requested

In [ ]:
elec_potential,_ = sx_el_potential3D_cell (Simulator['working_directory'])
pot,_,_,cell = fim.potential (Simulator).potential_cell ()

if 'z_ext' in Simulator:
    iz0 = Simulator['iz_ext_from']
    new_z_max=Simulator['z_ext']
    fig, pot_ext = extend_potential(elec_potential / fim.HARTREE_TO_EV, 
                                    iz0, pot, cell, z_max=new_z_max,
                                    izend=Simulator['izend'],dv_limit=1e-4,plotG=1)
    # copy extension from pot to elec_potential
    elec_ext = pot_ext[:,:,:,0] * fim.HARTREE_TO_EV
    elec_ext[:,:,0:iz0] = elec_potential[:,:,:iz0]
else:
    pot_ext = pot
    elec_ext = elec_potential

# Set up reader and FIM simulator

In [ ]:
# waves reader
waves_reader = sx_nc_waves_reader(Simulator['working_directory'] + "/waves.sxb")

# FIM simulator
fimsim=fim.FIM_simulations(Simulator,reader=waves_reader,V_total=pot_ext,V_elstat=elec_ext)

# Do the actual calculation for a single k-point

In [ ]:
fimsim.sum_single_k (Simulator['ik'])